In [4]:
import os
import json
from bs4 import BeautifulSoup
import re

def extract_text_and_title_from_html(html_filepath):
    """
    从指定的 HTML 文件中提取标题和正文文本。
    """
    try:
        with open(html_filepath, 'r', encoding='utf-8') as f:
            html_content = f.read()

        soup = BeautifulSoup(html_content, 'lxml') 

        title_tag = soup.find('title')
        title_string = title_tag.string if title_tag else None
        title = title_string.strip() if title_string else os.path.basename(html_filepath)
        title = title.replace('.html', '')

        content_tag = soup.find('content') or soup.find('div', class_='rich_media_content') or \
                      soup.find('article') or soup.find('main') or soup.find('body')

        if content_tag:
            text = content_tag.get_text(separator='\n', strip=True)
            return title, text
        else:
            print(f"警告：在文件 {html_filepath} 中未找到明确的正文标签。")
            return title, None 

    except FileNotFoundError:
        print(f"错误：文件 {html_filepath} 未找到。")
        return None, None
    except Exception as e:
        print(f"处理文件 {html_filepath} 时出错: {e}")
        return None, None

def split_text(text, max_chunks=2, chunk_size=2000):
    """
    将文本分割成最多指定块数的块。
    """
    if not text:
        return []

    total_length = len(text)
    
    # 如果长度小于chunk_size，直接返回
    if total_length <= chunk_size:
        return [text.strip()]

    # 计算应分割的块数，最多为max_chunks
    desired_chunks = min(max_chunks, (total_length + chunk_size - 1) // chunk_size)

    chunks = []
    start = 0

    # 每块的 size 取决于总长和需要的块数
    step_size = total_length // desired_chunks

    for i in range(desired_chunks):
        end = start + step_size
        if i == desired_chunks - 1:  # 确保最后一块包含剩余文本
            end = total_length
        chunks.append(text[start:end].strip())
        start = end

    return [chunk for chunk in chunks if chunk]  # 只返回非空块

def contains_blocked_words(text, blocked_words):
    """
    检查文本中是否包含任意屏蔽词。
    """
    pattern = re.compile('|'.join(map(re.escape, blocked_words)))
    return bool(pattern.search(text))

# --- 配置 ---
html_directory = './data/html/' 
output_json_path = './data/processed_data.json' 
CHUNK_SIZE = 2000  # 每个文本块的目标大小（字符数）
BLOCKED_WORDS = ['阅读原文', '点点赞', '爱心', '广告', '推广', '推荐阅读', '此内容为.*', '点击这里', '更多信息', '访问网站', '公众号', '暖雪', '暖雪小屋']

# --- 主处理逻辑 ---
all_data_for_milvus = []
file_count = 0
chunk_count = 0

print(f"开始处理目录 '{html_directory}' 中的 HTML 文件...")

os.makedirs(os.path.dirname(output_json_path), exist_ok=True)

html_files = [f for f in os.listdir(html_directory) if f.endswith('.html')]
print(f"找到 {len(html_files)} 个 HTML 文件。")

for filename in html_files:
    filepath = os.path.join(html_directory, filename)
    file_count += 1

    title, main_text = extract_text_and_title_from_html(filepath)

    if main_text:
        # 获取合并后的文本块
        chunks = split_text(main_text, max_chunks=2, chunk_size=CHUNK_SIZE)

        # 过滤掉含有屏蔽词的块
        for i, chunk in enumerate(chunks):
            if not contains_blocked_words(chunk, BLOCKED_WORDS):
                chunk_count += 1
                milvus_entry = {
                    "id": f"{filename}_{i}", 
                    "title": title or filename, 
                    "abstract": chunk, 
                    "source_file": filename, 
                    "chunk_index": i
                }
                all_data_for_milvus.append(milvus_entry)
            # else:
                # print(f"    警告：块 {i} 含有屏蔽词，已跳过。")
    else:
        print(f"    警告：未能从 {filename} 提取有效文本内容。")

print(f"\n处理完成。共处理 {file_count} 个文件，生成 {chunk_count} 个文本块。")

# --- 保存为 JSON ---
try:
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(all_data_for_milvus, f, ensure_ascii=False, indent=4)
    print(f"结果已保存到: {output_json_path}")
except Exception as e:
    print(f"错误：无法写入 JSON 文件 {output_json_path}: {e}")

开始处理目录 './data/html/' 中的 HTML 文件...
找到 34417 个 HTML 文件。
    警告：未能从 2018-06-13_向日葵儿童招聘啦！坐标北京 欢迎你的加入！.html 提取有效文本内容。
    警告：未能从 2018-10-01_分享图片.html 提取有效文本内容。
    警告：未能从 2019-01-01_分享图片.html 提取有效文本内容。
    警告：未能从 2019-02-09_分享图片.html 提取有效文本内容。
    警告：未能从 2020-01-24_分享图片.html 提取有效文本内容。
    警告：未能从 2020-04-04_分享图片.html 提取有效文本内容。
    警告：未能从 2020-08-07_我爱你，中国！.html 提取有效文本内容。
    警告：未能从 2020-09-08_感谢有你.html 提取有效文本内容。
    警告：未能从 2020-09-10_和你在一起！这是献给每个老师的歌.html 提取有效文本内容。
    警告：未能从 2021-01-05_科普一下——马军教授：血液系统疾病有哪几种？.html 提取有效文本内容。
    警告：未能从 2021-01-08_科普一下——马军教授：血液系统疾病的几大症状.html 提取有效文本内容。
    警告：未能从 2021-01-09_科普一下——马军教授：血液系统疾病治疗要点.html 提取有效文本内容。
    警告：未能从 2021-01-10_科普一下——马军教授：如何预防血液系统疾病.html 提取有效文本内容。
    警告：未能从 2021-01-12_科普一下——傅卫军教授：骨髓瘤整体治疗的检测.html 提取有效文本内容。
    警告：未能从 2021-01-13_科普一下——傅卫军教授：何为造血干细胞移植.html 提取有效文本内容。
    警告：未能从 2021-01-15_科普一下——傅卫军教授：异基因造血干细胞移植.html 提取有效文本内容。
    警告：未能从 2021-01-18_科普一下——傅卫军教授：骨髓瘤治疗的缓解程度.html 提取有效文本内容。
    警告：未能从 2021-01-19_科普一下——陈文明教授：骨髓瘤诊断标准之浆细胞形态.html 提取有效